## Setup

In [1]:
import numpy as np
import os
import pandas as pd
import shutil

In [2]:
cwd = '\\'.join((os.getcwd()).split('\\')[:-1])
cwd

'c:\\repos\\hm\\MAURICE'

## Test vari

In [3]:
f = open(os.path.join(cwd, 'test_models', 'm19_reach_mod.sfr'), 'a')
f.write('\n test test')
f.close()

In [4]:
import fortranformat as ff
header_line = ff.FortranRecordWriter('(A15, A15, A15)')
header_line.write(['x', 'y', 'z'])
line = ff.FortranRecordWriter('(3F15.3)')
line.write([1.0, 0.0, 0.5])
line.write([1.1, 0.1, 0.6])

'          1.100          0.100          0.600'

In [10]:
# Load SFR characteristics from Excel files and shapefile
sfr_data = os.path.join(cwd, 'test_models', 'm11_C1_sfr_data.xlsx')
# sfr_shp = gpd.read_file(os.path.join(cwd, 'test_models', 'SFR_ICALC2.shp'))

# Load general parameters (item 1)
it1 = pd.read_excel(sfr_data, sheet_name = 'ITEM1')

# Load reach data (item 2)
# reach_data = pd.read_excel(sfr_data, sheet_name = 'ITEM2b')
# reach_data = reach_data.merge(sfr_shp.loc[:, ['node', 'row', 'column', 'layer']], left_on = 'NRCH', right_on = 'node')
# reach_data = reach_data.drop(columns = 'NRCH')
# reach_data = reach_data.apply(pd.to_numeric)
# reach_data.columns = ['iseg', 'ireach', 'rchlen', 'strtop', 'slope',  'strthick',  'strhc1', 'node', 'k', 'i', 'j']
# reach_data = reach_data.loc[:,:].to_records(index = False)
# flopy adds 1 to node???
# reach_data.node = reach_data.node - 1

# Load item 5
it5 = pd.read_excel(sfr_data, sheet_name = 'ITEM5')

# Load segment data (item 6a)
segment_data = pd.read_excel(sfr_data, sheet_name = 'ITEM6abc')
segment_data.columns = [x.lower() for x in segment_data.columns]
segment_data.drop(columns = ['width1', 'width2'], inplace=True)
segment_data = segment_data.loc[:,:].to_records(index = False)
# drop width1, width2
# segment_data = {0: segment_data}

it6d = pd.read_excel(sfr_data, sheet_name = 'ITEM6d')
geom_data = {}
for seg in it6d.segment.unique():
    tool = it6d.loc[it6d.segment == seg, [f'v{i}' for i in range(1,9)]].to_numpy().copy()
    geom_data[int(seg)] = [tool[0].tolist(), tool[1].tolist()]
# geom_data = {0: geom_data}

# Set the unit number
unit_number = 27

# Load the data to be set up into the different stress periods
q = pd.read_csv(os.path.join(cwd, 'test_models', 'portate_segmenti.csv'))
q[q.isna()] = 0

In [11]:
df_segment_data = {}
for sp in range(1, 91+1):
    tool = q.loc[q.SP == sp, ['Qin', 'Sg_2', 'Sg_3', 'Sg_4', 'Sg_5', 'Sg_6', 'Sg_7']].values[0]
    tool = np.around(tool, 5)
    tool = [tool[0]] + [-x if x != 0 else 0 for x in tool[1:]]
    segment_data[0].flow = tool
    df_segment_data[sp] = segment_data[0].copy()

In [14]:
line = ff.FortranRecordWriter('(12F15.5)')
line.write([i for i in df_segment_data[1][0]])

In [53]:
line5 = ''.join([f'{i:10d}' for i in it5.values[0]])
print(line5)

         7         0         0         0


In [45]:
tool = df_segment_data[1][0]
line6a = f"{tool.nseg:10d}{tool.icalc:10d}{tool.outseg:10d}{tool.iupseg:10d}{tool.iprior:10d}{tool.nstrpts:10d}{tool.flow:10f}{tool.runoff:10d}{tool.etsw:10d}{tool.pptsw:10d}{tool.roughch:10f}{tool.roughbk:10f}"
line6a

'         1         2         2         0         0         0  1.185710         0         0         0  0.017000  0.017000'

In [44]:
xcpt = geom_data[0][1][0]
zcpt = geom_data[0][1][1]
line6d1 = ''.join([f'{x:10f}' for x in xcpt])
line6d2 = ''.join([f'{z:10f}' for z in zcpt])
print(line6d1)
print(line6d2)

  0.000000  2.390000  4.380000  4.730000  9.290000 15.260000 15.560000 19.730000
  3.290000  1.800000  0.200000  0.100000  0.000000  0.100000  0.200000  3.290000


## Scrittura del file .sfr tramite impostazione del formato

In [3]:
# create copy of original file
shutil.copyfile(os.path.join(cwd, 'test_models', 'm19_2_reach.sfr'), os.path.join(cwd, 'test_models', 'm19_2_reach_segment.sfr'))
shutil.copyfile(os.path.join(cwd, 'test_models', 'm19_2_reach.sfr'), os.path.join(cwd, 'test_models', 'm19_2_reach_segment_stazionario.sfr'))

# load copy
sfr = open(os.path.join(cwd, 'test_models', 'm19_2_reach_segment.sfr'), 'a')
sfr_staz = open(os.path.join(cwd, 'test_models', 'm19_2_reach_segment_stazionario.sfr'), 'a')

In [4]:
# load sfr data from excel
sfr_data = os.path.join(cwd, 'test_models', 'm19_sfr_data.xlsx')

# Load item 5
it5 = pd.read_excel(sfr_data, sheet_name = 'ITEM5')

# Load segment data (item 6a)
sd = pd.read_excel(sfr_data, sheet_name = 'ITEM6a')
sd.columns = [x.lower() for x in sd.columns]
# sd.drop(columns = ['width1', 'width2'], inplace=True)
sd = sd.loc[:,:].to_records(index = False)

# load geometry data (item 6d) 
it6d = pd.read_excel(sfr_data, sheet_name = 'ITEM6d')
geom_data = {}
for seg in it6d.segment.unique():
    tool = it6d.loc[it6d.segment == seg, [f'v{i}' for i in range(1,9)]].to_numpy().copy()
    geom_data[int(seg)] = [tool[0].tolist(), tool[1].tolist()]

# Load the data to be set up into the different stress periods
q = pd.read_csv(os.path.join(cwd, 'test_models', 'portate_segmenti_sp69.csv'))
q[q.isna()] = 0

In [60]:
# set number of stress periods
nsp = 69
nseg = 7

In [61]:
for sp in range(1, nsp+1):
    # extract discharge for selected stress period
    qs = q.loc[q.SP == sp, ['Qin', 'Sg_2', 'Sg_3', 'Sg_4', 'Sg_5', 'Sg_6', 'Sg_7']].values[0]
    qs = np.around(qs, 5)
    qs = [qs[0]] + [-x if x != 0 else 0 for x in qs[1:]]
    # add sp discharge to the segment data structure
    sd.flow = qs
    if sp == 1:
        line5 = '\n' + ''.join([f'{i:3d}' for i in it5.values[0]]) + '\n'
        sfr_staz.write(line5)
    else:
        line5 = ''.join([f'{i:3d}' for i in it5.values[0]]) + '\n'
    sfr.write(line5)
    for seg in range(0, nseg):
        # compose stress period lines
        
        line6a = f"{sd[seg].nseg:3d}{sd[seg].icalc:3d}{sd[seg].outseg:3d}{sd[seg].iprior:3d}{sd[seg].flow:10f}{sd[seg].runoff:10f}{sd[seg].etsw:10f}{sd[seg].pptsw:10f}{sd[seg].roughch:10f}{sd[seg].roughbk:10f}" + '\n'
        line6d1 = ''.join([f'{x:10f}' for x in geom_data[seg+1][0]]) + '\n'
        line6d2 = ''.join([f'{z:10f}' for z in geom_data[seg+1][1]]) + '\n'

        # write lines to file
        sfr.write(line6a)
        sfr.write(line6d1)
        sfr.write(line6d2)
        if sp == 1:
            sfr_staz.write(line6a)
            sfr_staz.write(line6d1)
            sfr_staz.write(line6d2)

# close file
sfr.close()
sfr_staz.close()

## Altro metodo: preparare file di input per GWV

In [5]:
# Load the data to be set up into the different stress periods
q = pd.read_csv(os.path.join(cwd, 'test_models', 'portate_segmenti_sp69.csv'))
q[q.isna()] = 0

tool = q[['SP', 'Qin'] + [f'Sg_{i}' for i in range(2,8)]]
tool = tool.melt(id_vars='SP')

tool.columns = ['SP', 'SEG', 'FLOW']
seg = {
    'Qin': 1,
    'Sg_2': 2,
    'Sg_3': 3,
    'Sg_4': 4,
    'Sg_5': 5,
    'Sg_6': 6,
    'Sg_7': 7,
}
tool.SEG = [seg[x] for x in tool.SEG]
tool['ICALC'] = 2

In [18]:
tool.to_csv(os.path.join(cwd, 'test_models', 'portate_segmenti_sp69_perGWV.csv'))